In [14]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

Import Tools

In [15]:
from tools.GetPayslipTool import get_payslip
from tools.GetReservationsTool import get_reservations
from tools.GetContractsTool import get_contracts
from tools.GetScheduleTool import get_schedule

Import State and Agents

In [16]:
from agents.agent_state import AgentState
from action_execution_agent import ActionExecutionAgent

Load Azure Models

In [17]:
from agents.llm_model import AzureModelProvider
provider = AzureModelProvider()

llm = provider.get_primary_model()
fast_llm = provider.get_light_model()

In [18]:
llm.invoke([SystemMessage(content="Ready to assist?")]).content

'Hello! How can I assist you today?'

Import Prompts

In [19]:
# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier_prompt.txt"), "r") as f:
    intent_classifier_prompt = f.read()

# Load the action execution prompt from the text file
with open(os.path.join(parent_dir, "prompts", "action_execution_prompt.txt"), "r") as f:
    action_execution_prompt = f.read()


## Agent Set-up

In [20]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

# Ensure AgentState is defined (as discussed previously)
# class AgentState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     next_action: str
#     retrieved_data: str
#     error: str

class ChatSupervisorAgent:
    def __init__(self, model, checkpointer=None):
        self.model = llm
        self.system = intent_classifier_prompt
        
        self.action_agent_instance = ActionExecutionAgent(
            llm_model=llm,
            system_prompt=action_execution_prompt
        ) 

        graph = StateGraph(AgentState)

        # --- 1. Define Nodes ---
        graph.add_node("classify_intent", self.classify_intent_node)
        graph.add_node("action_execution_agent", self.action_agent_instance.run)
        graph.add_node("information_retrieval_agent", self.information_retrieval_agent)
        graph.add_node("fallback_tool", self.fallback_tool)
        graph.add_node("answer_agent", self.answer_agent) # REQUIRED by architecture

        # --- 2. Define The Entry Point ---
        graph.set_entry_point("classify_intent")

        # --- 3. Define The Conditional Edge (The Router) ---
        graph.add_conditional_edges(
            "classify_intent", 
            self.route_intent, 
            {
                "ActionExecutionAgent": "action_execution_agent",
                "InformationRetrievalAgent": "information_retrieval_agent",
                "FallbackTool": "fallback_tool",
            }
        )

        # --- 4. Define Normal Edges (Enforcing ToV) ---
        graph.add_edge("action_execution_agent", "answer_agent")
        graph.add_edge("information_retrieval_agent", "answer_agent")
        graph.add_edge("fallback_tool", "answer_agent")
        
        # AnswerAgent finishes the run
        graph.add_edge("answer_agent", END)

        # 5. Compile
        self.graph = graph.compile(checkpointer=checkpointer)

    def classify_intent_node(self, state: AgentState):
        '''Reads the user query and classifies the primary action.'''
        messages = [SystemMessage(content=self.system)] + state['messages']
        response = self.model.invoke(messages)
        intent = response.content.strip()

        # Failsafe for valid routing
        valid_routes = ["ActionExecutionAgent", "InformationRetrievalAgent", "FallbackTool"]
        if intent not in valid_routes:
            print(f"Warning: Invalid intent '{intent}'. Defaulting to Fallback.")
            intent = "FallbackTool"

        print(f"Intent classified as: {intent}")
        
        # Return the update to the state
        return {"next_action": intent}

    def route_intent(self, state: AgentState):
        '''Reads the decision made by the intent classifier node'''
        return state["next_action"]

    # --- SPECIALISTS ---
    def action_execution_agent(self, state: AgentState):
        '''...'''
        print("---Start Action Execution Agent---")
        # Logic to call your tools...
        retrieved_data = ""
        
        
        # return {"retrieved_data": "Schedule for next week..."}
        return {"retrieved_data": retrieved_data}

    def information_retrieval_agent(self, state: AgentState):
        '''Retrieves information from the Sharepoint KnowledgeBase'''
        print("---Start Information Retrieval Agent---")
        # Logic to call RAG...
        return {"retrieved_data": "Sick leave policy details..."}

    def fallback_tool(self, state: AgentState):
        print("---Start Fallback Tool---")
        return {"error": "Could not resolve query."}

    # --- ANSWER AGENT (Final Response) ---
    def answer_agent(self, state: AgentState):
        print("---Start Answer Agent---")
        # return {"messages": [AIMessage(content="Final response to user")]}
        return {}

In [21]:
agent = ChatSupervisorAgent(model=llm)

In [22]:
# message = "what's my schedule for next week?"
message = "How can I call in sick?"
message = "What's my last payslip?"
# message = "What's Trump's first name?"

In [27]:
test_state = {
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

test_state

{'messages': [HumanMessage(content="What's my last payslip?", additional_kwargs={}, response_metadata={})],
 'candidate_id': '100102195',
 'employee_number': 'RE0036443',
 'next_action': '',
 'retrieved_data': '',
 'error_message': '',
 'attempts': 0,
 'max_retries': 3}

In [28]:
# Run
result = agent.graph.invoke(test_state)

Intent classified as: ActionExecutionAgent
---ACTION EXECUTION AGENT: Starting run()---
---Retrieved context: candidate_id=100102195, employee_number=RE0036443---
---Last message: What's my last payslip?...---
---Prepared 2 messages for LLM---
---Calling LLM with tools---
---LLM response received. Tool calls: 1---
---Executing Tool: get_payslip with args: {'candidate_id': '100102195', 'latest': True}---
---Tool execution completed---
---Tool result length: 120 characters---
---Start Answer Agent---


In [39]:
result["retrieved_data"]

"id=1223595 business_unit_id=10023 serial_number='25-46-001' frequency='W' entry_date='2025-11-19T00:12:52' amount=448.97"

In [30]:
agent.__getstate__

<function ChatSupervisorAgent.__getstate__()>